In [4]:
# 4-2

# =========================================================
# BERT（Bidirectional Encoder Representations from Transformers）の理論メモ
# ---------------------------------------------------------
# ■ 目的
#  - 文脈の双方向（左←→右）を同時に考慮した潜在表現 h ∈ R^H を得る。
#  - 事前学習（Pretraining）で一般的言語能力を獲得し、下流タスク（分類・系列ラベリング・QA等）に微調整（Fine-tuning）で適用。
#
# ■ 事前学習タスク
#  - MLM（Masked Language Modeling）:
#    入力トークン列 x の一部を [MASK] 等で隠し、p(x_masked | context) を最大化する。
#    近似的には、各マスク位置 i に対して softmax(W h_i) の対数尤度を最大化。
#  - NSP（Next Sentence Prediction）:
#    文 A の直後に文 B が来るかを 2 値分類（Tohoku BERT では採用）。RoBERTa 系では廃止例もある。
#
# ■ モデル（BertModel）の内部
#  - 入力埋め込み: token + position + segment（token_type）を加算して X ∈ R^{B×L×H} を作る。
#  - Transformer Encoder（L層）: 各層で Multi-Head Self-Attention + FFN。
#    • Attention は Scaled Dot-Product:
#        Q = XW_Q, K = XW_K, V = XW_V  (各 W_* ∈ R^{H×d_k})
#        scores = Q K^T / sqrt(d_k)
#        A = softmax(scores + mask)
#        head = A V
#      複数 head を結合し線形写像で次層へ。
#    • 計算量: O(L^2 H)（長文入力ほど二乗で増える）; メモリもほぼ O(L^2)。
#  - 出力:
#    • last_hidden_state: 形状 [B, L, H]。各トークン位置の文脈化表現。
#    • pooler_output: 形状 [B, H]。最終層の [CLS] ベクトルに線形＋tanh を適用（主に文分類の初期ベースライン）。
#    • hidden_states, attentions（オプション）: 各層の中間表現やアテンション行列を返す。
#
# ■ 日本語トークナイザ（BertJapaneseTokenizer）
#  - 典型的には形態素解析（MeCab/fugashi 等）で語彙単位に分割 → WordPiece に細分化。
#  - 特殊トークン: [CLS]（先頭）, [SEP]（区切り）, [PAD], [MASK], [UNK] など。
#  - 下流入力の慣例:
#      input_ids        : [B, L]（語彙ID）
#      attention_mask   : [B, L]（pad 0/非pad 1）
#      token_type_ids   : [B, L]（文 A=0 / 文 B=1）
#    これらを model(**batch) で渡す。
#
# ■ 実運用の注意
#  - 最大系列長: 多くの BERT は L ≤ 512。超える場合は分割や Longformer 等の検討。
#  - 学習: AdamW + 学習率ウォームアップ + 余弦減衰などが定番。weight decay は LayerNorm/バイアス除外。
#  - 乱数・再現性: torch.manual_seed, numpy.random.seed を固定。dropout の影響も考慮。
#  - 参照: Devlin et al., 2018; Tohoku NLP "cl-tohoku/bert-base-japanese" 系列
# =========================================================

import torch
from transformers import BertJapaneseTokenizer, BertModel

# ---------------------------------------------------------
# 依存関係の注記（インストール時の目安）
#  - transformers >= 4.x
#  - fugashi, ipadic（や unidic-lite）: BertJapaneseTokenizer の形態素解析で利用されることが多い
#    例) pip install transformers fugashi ipadic
# ---------------------------------------------------------

# ---------------------------------------------------------
# （参考）実行例：事前学習済み日本語BERTの読み込み
#   ※ 下の例は「使い方の目安」を示すためのコメントであり、実行は任意です。
#
# # 1) 日本語BERTのトークナイザ／モデルをロード
# tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
# model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
# model.eval()  # 推論時は eval モードで dropout 等を停止
#
# # 2) テキストを ID 列へ変換
# #   return_tensors="pt" で PyTorch テンソルを自動作成
# batch = tokenizer(
#     ["今日は良い天気ですね。", "しかし少し風が強いかもしれません。"],
#     padding=True, truncation=True, max_length=128, return_tensors="pt"
# )
# # batch: dict で input_ids/attention_mask/token_type_ids を含む
# # 形状: input_ids.shape == [B, L]
#
# # 3) モデルへ前向き計算
# with torch.no_grad():
#     outputs = model(**batch, output_attentions=False, output_hidden_states=False)
#
# # 4) 出力の形状
# # outputs.last_hidden_state: [B, L, H]  各トークン位置の表現
# # outputs.pooler_output   : [B, H]      [CLS] 由来（文表現のベースライン）
#
# # 5) 用途の例
# # - 文分類: pooler_output か、[CLS] の last_hidden_state[:, 0, :] を線形分類器へ
# # - トークン分類（固有表現抽出など）: last_hidden_state を各位置で線形＋softmax
# # - QA: [CLS]/[SEP] を含むペア入力で start/end スパンを回帰
#
# # 6) 理論と設計の対応
# # - attention_mask は pad 位置へのソフトマックスを -∞ マスクで抑制（scores に加算）
# # - token_type_ids は NSP/ペア入力の区別（A:0, B:1）。単文なら全 0 でよい。
# # - 学習では MLM/NSP（または SOP）に対応するヘッドを別途持つ（BertForPreTraining 等）。
# ---------------------------------------------------------

In [5]:
# 4-3

# ---------------------------------------------------------
# 理論メモ：Whole Word Masking（WWM）版日本語BERTを使う理由
# ---------------------------------------------------------
# ◇ 標準MLM vs. Whole Word Masking
#  - 標準のMLM（WordPiece単位の隠し方）では、語が「サブワード」に分割された場合、
#    その一部のサブワードだけが[MASK]になることが多い。
#  - WWMは「同じ語（= 形態素→WordPieceに分解される前の“語”）」に属する全サブワードを
#    まとめて同時にマスクする。これによりモデルは「語レベルの完形復元」を学ぶ圧力が強まり、
#    文脈一貫性や語彙レベルの表現獲得に寄与する（サブワード断片の当て推量を減らす）。
#  - 直感的には、より強い隠し課題（harder pretext task）→汎化に寄与する可能性。
#    ただし学習難易度は上がりうる（学習安定化はデータとハイパラ次第）。
#
# ◇ 日本語での影響
#  - 日本語は語境界が空白で明示されないため、形態素解析（例：MeCab + 辞書）で語単位を推定→
#    さらにWordPieceで細分化、という段階を踏む。この「語」単位でマスクを丸ごと掛けるのがWWM。
#  - 形態素解析の精度や辞書選択（ipadic / unidic-lite 等）に依存する側面がある。
#
# ◇ 実務上の注意
#  - このモデル名 'tohoku-nlp/bert-base-japanese-whole-word-masking' は、
#    東北大NLPが公開している日本語BERT（WWM版）の重み・語彙を指す。
#  - Tokenizer/Modelは対応するペアを使う（語彙の不一致はIDずれ→性能劣化）。
#  - 依存関係：transformers、fugashi、ipadic（またはunidic-lite）等を事前にインストール。
#    例：pip install transformers fugashi ipadic
# ---------------------------------------------------------

model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"  # WWM版日本語BERT（語レベルでの一括マスキングで事前学習）
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# （参考）挙動確認スニペット：※必要なら実行
# text = "深層学習の事前学習モデルBERTは強力です。"
# enc = tokenizer(
#     text,
#     return_tensors="pt",      # PyTorchテンソルで返す
#     padding="max_length",     # 最大長にパディング（推論/バッチ化の整列用）
#     truncation=True,          # 上限長超過時に切り詰め
#     max_length=32
# )
# # enc["input_ids"].shape == [1, 32], enc["attention_mask"].shape == [1, 32]
# # WWMは事前学習時のマスキング戦略であり、推論時のトークナイズ結果（input_ids自体）が
# # 直接変わるわけではない点に注意（学習済み表現に差が出る）。
#
# # なお、下流タスクでは対応するモデル（例：BertModel/BertForSequenceClassification など）を
# # 同じ `model_name` から読み出し、tokenizerとペアで使うことが重要。

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [6]:
# 4-4 ～ 4-6
# =========================================================
# 理論補足：BertJapaneseTokenizerのトークナイズ動作（日本語×WordPiece）
# ---------------------------------------------------------
# ■ 概要
#  - BertJapaneseTokenizer は「形態素解析 → WordPiece 分割」という2段階でトークン化する。
#    1) 形態素解析：文を語（形態素）へ分割（MeCab + 辞書 ipadic / unidic-lite 等）
#    2) WordPiece：各「語」を語彙に基づいて更にサブワードへ分割（語彙外は細分化、最悪 [UNK]）
#  - 返り値 tokenizer.tokenize(text) は特殊トークンを付与しない素のサブワード列（list[str]）。
#    例：['自', '然', '言', '語', '処', '理'] のような字単位、あるいは ['自然', '言語', '処理'] など
#    語彙と辞書の組み合わせにより変動する（＝「必ずこの分割になる」とは限らない）。
#
# ■ Whole Word Masking（WWM）との関係
#  - 本トークナイザ自体は「推論・前処理」でのサブワード列を返すのみ。
#  - WWM は「事前学習時のマスク戦略」で、同一“語”（形態素に相当）に属するサブワードを
#    まとめて同時にマスクする。推論時の tokenize 結果は WWM の有無で直接は変わらない。
#
# ■ 文ごとの観察ポイント
#  (A) 「明日は自然言語処理の勉強をしよう。」
#    - 「自然言語処理」は複合名詞。形態素解析の辞書設定により
#      「自然/言語/処理」や「自然言語/処理」等に分かれうる。
#    - その後、WordPiece がさらに細分化（語彙にあればそのまま、無ければ細切れ）。
#  (B) 「明日はマシンラーニングの勉強をしよう。」
#    - カタカナ複合語。「マシン」「ラーニング」に分かれる場合や
#      「マシンラーニング」単語→サブワード細分の可能性がある。
#  (C) 「機械学習を中国語にすると机器学习だ。」
#    - 末尾の「机器学习」は中国語（簡体字）。日本語辞書では未知語扱いになりやすく、
#      WordPiece が文字単位や部分列へ分解、最悪 [UNK] へフォールバックしうる。
#    - BERT の日本語語彙は CJK を広く含むため [UNK] にならず字単位分割で拾えることも多い。
#
# ■ 実務上のTips
#  - 再現性：辞書種別（ipadic / unidic-lite）・バージョン、transformers のバージョンで分割が変わる。
#  - 下流処理：tokenize → convert_tokens_to_ids → [CLS]/[SEP] 付与 → attention_mask 作成、が一般手順。
#  - 形状把握：下流モデル入力は通常 encode_plus / __call__（return_tensors="pt"）を使うのが安全。
# =========================================================

# 既に 4-3 で `tokenizer` を作っている想定。
# セッション直実行でも動くように保険として用意（未定義なら初期化）。
try:
    tokenizer  # 変数が存在するかチェック
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 4-4：語彙・辞書により複合名詞「自然言語処理」の扱いが変動しうる点に注目
tokens_44 = tokenizer.tokenize("明日は自然言語処理の勉強をしよう。")
# デバッグ表示（必要ならコメント解除）
# print(tokens_44)

# 4-5：カタカナ複合語の分割（「マシン」「ラーニング」等）やサブワード接頭辞（##）の有無に注目
tokens_45 = tokenizer.tokenize("明日はマシンラーニングの勉強をしよう。")
# print(tokens_45)

# 4-6：簡体字中国語「机器学习」の未知語挙動（字単位分割や [UNK] フォールバック）に注目
tokens_46 = tokenizer.tokenize("機械学習を中国語にすると机器学习だ。")
# print(tokens_46)

# （参考）ID 列へ変換して下流モデル入力へ繋ぐ例：
# enc_44 = tokenizer('明日は自然言語処理の勉強をしよう。', return_tensors='pt')
# # enc_44 は input_ids / attention_mask / token_type_ids を含む辞書
# # enc_44['input_ids'].shape == [1, L]